In [1]:
import cv2
import numpy as np
from math import sqrt
from heapq import nsmallest
from heapq import nlargest

In [40]:
img_train = cv2.imread("train.png", cv2.IMREAD_GRAYSCALE)
img_test = cv2.imread("test.png", cv2.IMREAD_GRAYSCALE)
# cv2.imshow("train", img_train)
# cv2.imshow("test", img_test)

In [41]:
print(img_test)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [42]:
row_img = 50
col_img = 50
tot = 250
img_per_baris = np.vsplit(img_train, row_img)
cells = []
for row in img_per_baris:
    per_digit = np.hsplit(row, col_img)
    for cell in per_digit:
        cell = cell.ravel()
        cells.append(cell)
# print(cells[0])

In [43]:
# cells[0]

In [44]:
k = np.arange(0,10)
label = np.repeat(k, 250)

In [45]:
len(label)

2500

In [46]:
img_test = np.vsplit(img_test,50)
test_cells = []
for d in img_test:
    d = d.ravel()
    test_cells.append(d)

# KNN

In [47]:
# KNN
cells = np.array(cells, dtype=np.float32)
test_cells = np.array(test_cells, dtype=np.float32)
knn = cv2.ml.KNearest_create()
knn.train(cells, cv2.ml.ROW_SAMPLE, label)
ret, result, neighbours, dist = knn.findNearest(test_cells, k=3)

# print(result)

In [48]:
a = np.arange(10)
t = np.repeat(a, 5)
count = 0
for i in range (len(t)) :
    if (result[i] == t[i]) :
        count += 1
acc = (count / 50) * 100
print(acc, "%")

90.0 %


# MKNN

In [51]:
#Euclidean per digit
d_between_digit = []
for digit_home in cells:
    d_per_digit = []
    for digit_away in cells:
        total = 0
        for i in range(400):
            sum = (digit_home[i] - digit_away[i])**2
            total = total + sum
        d_per_digit.append(sqrt(total))
    d_between_digit.append(d_per_digit)

In [52]:
#Validasi data training, 3 minimum data euclidean
h = 10
validity = []
for lbl, digit in zip(label, d_between_digit):
    smallest = nsmallest(h+1, digit)
    id_smallest = []
    for num in smallest:
        id_smallest.append(digit.index(num))
    count = 0
    for i in range(h):
        if lbl == label[id_smallest[i+1]]:
            count += 1
    validity.append(count/h)

In [53]:
# for i in range(len(validity)):
#     print(i, " ", validity[i])

In [54]:
#Euclidean data test
d_test_to_train = []
for digit_home in test_cells:
    d_per_digit = []
    for digit_away in cells:
        total = 0
        for i in range(400):
            sum = (digit_home[i] - digit_away[i])**2
            total = total + sum
        d_per_digit.append(sqrt(total))
    d_test_to_train.append(d_per_digit)

In [55]:
# d_test_to_train[0]

In [56]:
#Weight Voting
weight_vote = []
for digit in d_test_to_train:
    w = []
    for dist, valid in zip(digit, validity):
        w.append(valid*(1/(dist+0.5)))
    weight_vote.append(w)

In [57]:
# len(weight_vote[0])

In [58]:
k=3
predict = []
for digit in weight_vote:
    largest = nlargest(k, digit)
    largest_class = []
    for num in largest:
        largest_class.append(label[digit.index(num)])
    predict.append(max(set(largest_class), key = largest_class.count))

In [59]:
# predict

In [60]:
count = 0
a = np.arange(10)
t = np.repeat(a, 5)
for num, lbl in zip(predict, t):
    if num == lbl:
        count += 1
accuracy_MKNN = count/len(predict)
print(accuracy_MKNN * 100)

88.0
